In [1]:
import torch
from model.UNET import *
import numpy as np
import cv2
import torchvision.transforms as transforms
from PIL import Image
from matplotlib import pyplot as plt

In [2]:

#Save scripted model
model=UNet(n_channels=3,n_classes=3)
# model.load_state_dict(torch.load('D:\\UBB\\CVDL\\Project\\model\\model6.pth'))
model.load_state_dict(torch.load('D:\\UBB\\CVDL\\Project\\checkpoints\\best_run.pth'))


model.eval()
#example = torch.rand(1, 3, 256, 256)
scripted_model=torch.jit.script(model)

# scripted_model.save("D:\\UBB\\CVDL\\Project\\scripted\\scripted_model6.pt")
scripted_model.save("D:\\UBB\\CVDL\\Project\\scripted\\scripted_model7.pt")



In [3]:
model = torch.jit.load("D:\\UBB\\CVDL\\Project\\scripted\\scripted_model6.pt")
def get_mask(img):
    model.eval()
    img=img.unsqueeze(0)
    with torch.no_grad():
        mask=model(img)
        mask=mask.squeeze(0).detach()
        mask=mask.permute(1,2,0).numpy()
        mask = (mask - mask.min()) / (mask.max() - mask.min()) * 255
        return mask.astype(np.uint8)
    
def get_mask2(img):
    model.eval()
    img=img.unsqueeze(0)
    with torch.no_grad():
        mask=model(img)
        mask=mask.squeeze(0).detach()
        mask=mask.permute(1,2,0)
        return mask
    
def blur_background(img):
    transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor()
    ])
    img = Image.fromarray(np.uint8(img))
    img = transform(img)
    
    mask=get_mask2(img)
    mask=mask.numpy()
    mask=mask.argmax(2)

    blurred_image=np.copy(img)
    blurred_image=blurred_image.astype(float)
    blurred_image=np.transpose(blurred_image,(1,2,0))

    background_mask = mask==2

    blurred_background=cv2.GaussianBlur(blurred_image,(51,51),0)
    blurred_image[background_mask]=blurred_background[background_mask]
    return blurred_image

    
    


In [4]:
import gradio as gr

demo = gr.Interface(fn=blur_background,
                    inputs=gr.Image(),
                    #inputs=gr.Image(sources=["webcam"]),
                    outputs="image",
                    title="Image background blur!")

demo.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
